Solving Rock-Paper-Scissors using CFR

Paper: http://modelai.gettysburg.edu/2013/cfr/cfr.pdf

In [79]:
import numpy as np
import random
import math
from itertools import combinations_with_replacement

In [80]:
NUM_ACTIONS = 3 # 3 Actions, R, P, S
ROCK = 0
PAPER = 1
SCISSORS = 2

In [81]:
# Pick mixed action according to probability given
def get_action(strategy):
    return np.random.choice(np.arange(NUM_ACTIONS), p=strategy)

In [82]:
# Rock-Paper-Scissors Player class
class RPSPlayer:
    # Can initialize with given strategy
    def __init__(self, strategy_sum=np.zeros(NUM_ACTIONS)):
        self.regret_sum = np.zeros(NUM_ACTIONS) # Cumulative regret table, 0, 0, 0 for R, P, S
        self.strategy_sum = strategy_sum # Cumulative strategy table

    # Trains given number of iterations, by calculating utility, updating sum values.
    def self_train(self, opponent_strategy, iterations=10000):
        action_utility = np.zeros(NUM_ACTIONS)
        for _ in range(iterations):
            strategy = self.get_strategy()
            self_action = get_action(strategy)
            opponent_action = get_action(opponent_strategy)
            
            action_utility[opponent_action] = 0;
            action_utility[(opponent_action + 1) % 3] = 1
            action_utility[(opponent_action - 1) % 3] = -1

            for action in range(NUM_ACTIONS):
                self.regret_sum[action] += action_utility[action] - action_utility[self_action]

    # Gets strategy based on regret sum table
    def get_strategy(self):
        strategy = np.zeros(NUM_ACTIONS) # Strategy table
        normalizing_sum = 0
        for action in range(NUM_ACTIONS):
            strategy[action] = self.regret_sum[action] if (self.regret_sum[action] > 0) else 0
            normalizing_sum += strategy[action]
        for action in range(NUM_ACTIONS):
            if normalizing_sum > 0:
                strategy[action] /= normalizing_sum
            else:
                strategy[action] = 1.0 / NUM_ACTIONS
            self.strategy_sum[action] += strategy[action]
        return strategy

    # Total average strategy, to be used after minimizing regret over many iterations.
    # Uses strategy_sum to normalize, instead of regret_sum
    def get_average_strategy(self):
        average_strategy = np.zeros(NUM_ACTIONS)
        normalizing_sum = sum(self.strategy_sum)
        for action in range(NUM_ACTIONS):
            if normalizing_sum > 0:
                average_strategy[action] = self.strategy_sum[action] / normalizing_sum
            else:
                average_strategy[action] = 1.0 / NUM_ACTIONS
        return average_strategy

In [83]:
# Method to print strategy nicely
def format_strategy(strategy):
    return "Rock %: " + str(strategy[0]) + "\nPaper %: " + str(strategy[1]) + "\nScissors %: " + str(strategy[2]) + "\n"

In [85]:
# Exercise: RPS Equilibrium

unbalanced_strategy = np.array([3, 4, 5])
random_strategy = np.random.dirichlet(alpha=[1, 1, 1]) # Method to get random strategy (sums to 1)
random_strategy2 = np.random.dirichlet(alpha=[1, 1, 1])
player1 = RPSPlayer(random_strategy)
player2 = RPSPlayer(random_strategy2)

# Epoch #'s to print current strategies, to see as they develop
data_points = (0, 10, 100, 10000)

# View the initial strategies, before regret min
print("Initial strategies:\n")
print("Player 1:\n", format_strategy(player1.get_average_strategy()))
print("Player 2:\n", format_strategy(player2.get_average_strategy()))

for epoch in range(100000):
    # Set iterations to one in self training, so players can update after each.
    # This minimizes magnitude of walk away from equilibrium, in this case (0.3-, 0.3-, 0.3-)
    player2_strategy = player2.get_strategy()
    player1.self_train(player2_strategy, iterations=1)
    player1_strategy = player1.get_strategy()
    player2.self_train(player1_strategy, iterations=1)
    if epoch in data_points:
        print(f"----- After {epoch} iterations: ----- \n")
        print("Player 1 Strategy:\n", format_strategy(player1.get_average_strategy()))
        print("Player 2 Strategy:\n", format_strategy(player2.get_average_strategy()))


Initial strategies:

Player 1:
 Rock %: 0.2256183852875299
Paper %: 0.5776567715577962
Scissors %: 0.196724843154674

Player 2:
 Rock %: 0.016845839974561315
Paper %: 0.4642607049720584
Scissors %: 0.5188934550533802

----- After 0 iterations: ----- 

Player 1 Strategy:
 Rock %: 0.519650572873621
Paper %: 0.30366336829704316
Scissors %: 0.17668605882933577

Player 2 Strategy:
 Rock %: 0.22783750221374266
Paper %: 0.37697579054624164
Scissors %: 0.39518670724001564

----- After 10 iterations: ----- 

Player 1 Strategy:
 Rock %: 0.3882083908027014
Paper %: 0.26086430407739203
Scissors %: 0.3509273051199065

Player 2 Strategy:
 Rock %: 0.4293038563964095
Paper %: 0.22791471664129723
Scissors %: 0.3427814269622933

----- After 100 iterations: ----- 

Player 1 Strategy:
 Rock %: 0.3859682784097254
Paper %: 0.2191536893335403
Scissors %: 0.39487803225673423

Player 2 Strategy:
 Rock %: 0.37118528115143257
Paper %: 0.2942602141651963
Scissors %: 0.33455450468337117

----- After 10000 iteratio

In [86]:
# Exercise: Colonel Blotto
# Game with N battlefields, S soldiers. 2 Teams. Team that captures most battlefields wins.
# A team captures a battlefield if they send more soldiers than the opponent.
# A team can send any number of soldiers to each battlefield, including 0, but must sum < S.
# Will be solving for N = 3, S = 5
NUM_BATTLEFIELDS = 3
NUM_SOLDIERS = 5

# A Pure strategy would be any ordered set: (S1, S2, S3), Sum(S1-3) < 5, 0 <= Sn <= 5
# This is any permutation of BBSSSSS, so 7 choose 2. 21 total pure strategies.
NUM_ACTIONS = math.comb(NUM_BATTLEFIELDS + NUM_SOLDIERS - 1, NUM_BATTLEFIELDS - 1)


ALL_STRATEGIES = combinations_with_replacement(range(NUM_BATTLEFIELDS), NUM_SOLDIERS)
ALL_STRATEGIES = [list([combo.count(i) for i in range(NUM_BATTLEFIELDS)]) for combo in ALL_STRATEGIES]

PURE_STRATEGIES = {count: strat for count, strat in enumerate(ALL_STRATEGIES)}

# Method that returns the utility (-1, 0, 1) for strategy1
def strategy_utility(strategy1, strategy2):
    strategy1 = PURE_STRATEGIES[strategy1]
    strategy2 = PURE_STRATEGIES[strategy2]
    wins = 0
    for battlefield in range(NUM_BATTLEFIELDS):
        soldier_difference = strategy1[battlefield] - strategy2[battlefield]
        if soldier_difference != 0:
            wins += soldier_difference / abs(soldier_difference)
    return wins if wins == 0 else wins / abs(wins)

In [87]:
# War Commander class
class WarCommander:
    # Can initialize with given strategy
    def __init__(self, strategy_sum=np.zeros(NUM_ACTIONS)):
        self.regret_sum = np.zeros(NUM_ACTIONS) # Cumulative regret table for each pure action
        self.strategy_sum = strategy_sum # Cumulative strategy table

    # Trains given number of iterations, by calculating utility, updating sum values.
    def self_train(self, opponent_strategy, iterations=1):
        action_utility = np.zeros(NUM_ACTIONS)
        for _ in range(iterations):
            strategy = self.get_strategy()
            self_action = get_action(strategy)
            opponent_action = get_action(opponent_strategy)

            for action in range(NUM_ACTIONS):
                action_utility[action] = strategy_utility(action, opponent_action)

            for action in range(NUM_ACTIONS):
                self.regret_sum[action] += action_utility[action] - action_utility[self_action]

    # Gets strategy based on regret sum table
    def get_strategy(self):
        strategy = np.zeros(NUM_ACTIONS) # Strategy table
        normalizing_sum = 0
        for action in range(NUM_ACTIONS):
            strategy[action] = self.regret_sum[action] if (self.regret_sum[action] > 0) else 0
            normalizing_sum += strategy[action]
        for action in range(NUM_ACTIONS):
            if normalizing_sum > 0:
                strategy[action] /= normalizing_sum
            else:
                strategy[action] = 1.0 / NUM_ACTIONS
            self.strategy_sum[action] += strategy[action]
        return strategy

    # Total average strategy, to be used after minimizing regret over many iterations.
    # Uses strategy_sum to normalize, instead of regret_sum
    def get_average_strategy(self):
        average_strategy = np.zeros(NUM_ACTIONS)
        normalizing_sum = sum(self.strategy_sum)
        for action in range(NUM_ACTIONS):
            if normalizing_sum > 0:
                average_strategy[action] = self.strategy_sum[action] / normalizing_sum
            else:
                average_strategy[action] = 1.0 / NUM_ACTIONS
        return average_strategy

In [88]:
# Method to print strategy nicely
def format_strategy(strategy):
    strStrat = ''
    for num in range(NUM_ACTIONS):
        if strategy[num] < 1/(NUM_ACTIONS * 2):
            continue
        else:
            strStrat += 'Chance of playing: ' + str(PURE_STRATEGIES[num]) + ' is: %' + str(strategy[num]) + '\n'
    return strStrat

In [89]:
# Exercise: Colonel Blotto Equilibrium

random_strategy = np.random.dirichlet(alpha=np.ones(NUM_ACTIONS)) # Method to get random strategy (sums to 1)
random_strategy2 = np.random.dirichlet(alpha=np.ones(NUM_ACTIONS))
player1 = WarCommander(random_strategy)
player2 = WarCommander(random_strategy2)

# Epoch #'s to print current strategies, to see as they develop
data_points = (0, 10, 100, 1000, 10000)

# View the initial strategies, before regret min
print("Initial strategies:\n")
print("Player 1:\n", format_strategy(player1.get_average_strategy()))
print("Player 2:\n", format_strategy(player2.get_average_strategy()))

for epoch in range(100001):
    # Set iterations to one in self training, so players can update after each.
    # This minimizes magnitude of walk away from equilibrium, in this case (0.3-, 0.3-, 0.3-)
    player2_strategy = player2.get_strategy()
    player1.self_train(player2_strategy, iterations=1)
    player1_strategy = player1.get_strategy()
    player2.self_train(player1_strategy, iterations=1)
    if epoch in data_points:
        print(f"----- After {epoch} iterations: ----- \n")
        print("Player 1 Strategy:\n", format_strategy(player1.get_average_strategy()))
        print("Player 2 Strategy:\n", format_strategy(player2.get_average_strategy()))


Initial strategies:

Player 1:
 Chance of playing: [3, 0, 2] is: %0.06532137461927304
Chance of playing: [2, 3, 0] is: %0.07990176977132271
Chance of playing: [2, 2, 1] is: %0.1535871643027135
Chance of playing: [2, 1, 2] is: %0.03655802594277305
Chance of playing: [2, 0, 3] is: %0.11159410028370344
Chance of playing: [1, 3, 1] is: %0.06115838743617737
Chance of playing: [1, 0, 4] is: %0.12911676964316685
Chance of playing: [0, 5, 0] is: %0.02855516438884167
Chance of playing: [0, 4, 1] is: %0.043103685649046326
Chance of playing: [0, 3, 2] is: %0.12828665969984182
Chance of playing: [0, 2, 3] is: %0.028734767944172963
Chance of playing: [0, 0, 5] is: %0.038238088039908205

Player 2:
 Chance of playing: [4, 1, 0] is: %0.07191511403289813
Chance of playing: [4, 0, 1] is: %0.03720438422773986
Chance of playing: [3, 2, 0] is: %0.08899512684649585
Chance of playing: [2, 3, 0] is: %0.028314959562736733
Chance of playing: [2, 2, 1] is: %0.042363899039388
Chance of playing: [2, 1, 2] is: %0.0

CFR Min Stars Here! Kuhn Poker First!

In [90]:
# First, some data definitions
from sortedcontainers import SortedDict
import random

PASS = 0
BET = 1
NUM_ACTIONS = 2 # Bet, Pass

In [91]:
class KuhnNode:
    def __init__(self):
        self.infoSet = ""
        self.regretSum = [0.0] * NUM_ACTIONS
        self.strategy = [0.0] * NUM_ACTIONS
        self.strategySum = [0.0] * NUM_ACTIONS

    def getStrategy(self, realizationWeight):
        normalizingSum = 0
        for action in range(NUM_ACTIONS):
            if self.regretSum[action] > 0:
                self.strategy[action] = self.regretSum[action]
            else:
                self.strategy[action] = 0
            normalizingSum += self.strategy[action]
        for action in range(NUM_ACTIONS):
            if normalizingSum > 0:
                self.strategy[action] /= normalizingSum
            else:
                self.strategy[action] = 1.0 / NUM_ACTIONS
            self.strategySum[action] += realizationWeight * self.strategy[action]
        return self.strategy

    def getAverageStrategy(self):
        averageStrategy = [0.0] * NUM_ACTIONS
        normalizingSum = 0.0
        for action in range(NUM_ACTIONS):
            normalizingSum += self.strategySum[action]
        for action in range(NUM_ACTIONS):
            if normalizingSum > 0:
                averageStrategy[action] = self.strategySum[action] / normalizingSum
            else:
                averageStrategy[action] = 1.0 / NUM_ACTIONS
        return averageStrategy

    def __str__(self):
        avgStratsNice = [round(num, 4) for num in self.getAverageStrategy()]
        return f"{self.infoSet:>4}: Pass: {avgStratsNice[0]} Bet: {avgStratsNice[1]}"

In [92]:
def cfr(cards, history, p0, p1):
    plays = len(history)
    player = plays % 2
    opponent = 1 - player

    # Getting return payoffs for terminal states
    if (plays > 1):
        terminalPass = history[plays - 1] == 'p'
        doubleBet = history[plays - 2 : plays] == ("bb")
        isPlayerCardHigher = cards[player] > cards[opponent]
        if terminalPass:
            if history == ("pp"):
                return 1 if isPlayerCardHigher else -1
            else:
                return 1
        elif doubleBet:
            return 2 if isPlayerCardHigher else -2

    infoSet = str(cards[player]) + history

    # Get info set node, or create it
    node = nodeMap.get(infoSet)
    if node == None:
        node = KuhnNode()
        node.infoSet = infoSet
        nodeMap[infoSet] = node

    # Recursively call cfr with additional history and probability for each action
    strategy = node.getStrategy(p0 if player == 0 else p1)
    util = [0.0] * NUM_ACTIONS
    nodeUtil = 0
    for action in range(NUM_ACTIONS):
        nextHistory = history + ("p" if action == 0 else "b")
        if player == 0:
            util[action] = - cfr(cards, nextHistory, p0 * strategy[action], p1)
        else:
            util[action] = - cfr(cards, nextHistory, p0, p1 * strategy[action])
        nodeUtil += strategy[action] * util[action]

    # Compute and accumulate cfr for each action
    for action in range(NUM_ACTIONS):
        regret = util[action] - nodeUtil
        node.regretSum[action] += (p1 if player == 0 else p0) * regret

    return nodeUtil

In [93]:
class KuhnTrainer:
    def __init__(self):
        self.nodeMap = dict()
    
    def train(self, iterations):
        cards = list(range(1, 10))
        util = 0.0
        for iteration in range(iterations):
            random.shuffle(cards)
            util += self.cfr(cards, "", 1, 1)
        print("Average game value: ", util / iterations)
        for node in sorted(self.nodeMap.values(), key=(lambda node: node.infoSet)):
            print(node)

    def cfr(self, cards, history, p0, p1):
        plays = len(history)
        player = plays % 2
        opponent = 1 - player
    
        # Getting return payoffs for terminal states
        if (plays > 1):
            terminalPass = history[plays - 1] == 'p'
            doubleBet = history[plays - 2 : plays] == ("bb")
            isPlayerCardHigher = cards[player] > cards[opponent]
            if terminalPass:
                if history == ("pp"):
                    return 1 if isPlayerCardHigher else -1
                else:
                    return 1
            elif doubleBet:
                return 2 if isPlayerCardHigher else -2
    
        infoSet = str(cards[player]) + history
    
        # Get info set node, or create it
        node = self.nodeMap.get(infoSet)
        if node == None:
            node = KuhnNode()
            node.infoSet = infoSet
            self.nodeMap[infoSet] = node
    
        # Recursively call cfr with additional history and probability for each action
        strategy = node.getStrategy(p0 if player == 0 else p1)
        util = [0.0] * NUM_ACTIONS
        nodeUtil = 0
        for action in range(NUM_ACTIONS):
            nextHistory = history + ("p" if action == 0 else "b")
            if player == 0:
                util[action] = - self.cfr(cards, nextHistory, p0 * strategy[action], p1)
            else:
                util[action] = - self.cfr(cards, nextHistory, p0, p1 * strategy[action])
            nodeUtil += strategy[action] * util[action]
    
        # Compute and accumulate cfr for each action
        for action in range(NUM_ACTIONS):
            regret = util[action] - nodeUtil
            node.regretSum[action] += (p1 if player == 0 else p0) * regret
    
        return nodeUtil


In [70]:
trainer = KuhnTrainer()
trainer.train(1000000)

Average game value:  -0.06348452015938917
   1: Pass: 0.7933 Bet: 0.2067
  1b: Pass: 1.0 Bet: 0.0
  1p: Pass: 0.0023 Bet: 0.9977
 1pb: Pass: 1.0 Bet: 0.0
   2: Pass: 0.4167 Bet: 0.5833
  2b: Pass: 1.0 Bet: 0.0
  2p: Pass: 0.6739 Bet: 0.3261
 2pb: Pass: 1.0 Bet: 0.0
   3: Pass: 0.9913 Bet: 0.0087
  3b: Pass: 0.8022 Bet: 0.1978
  3p: Pass: 0.9998 Bet: 0.0002
 3pb: Pass: 0.7335 Bet: 0.2665
   4: Pass: 0.9908 Bet: 0.0092
  4b: Pass: 0.4392 Bet: 0.5608
  4p: Pass: 1.0 Bet: 0.0
 4pb: Pass: 0.3889 Bet: 0.6111
   5: Pass: 1.0 Bet: 0.0
  5b: Pass: 0.4526 Bet: 0.5474
  5p: Pass: 0.9998 Bet: 0.0002
 5pb: Pass: 0.3854 Bet: 0.6146
   6: Pass: 0.8039 Bet: 0.1961
  6b: Pass: 0.0004 Bet: 0.9996
  6p: Pass: 0.0241 Bet: 0.9759
 6pb: Pass: 0.0 Bet: 1.0
   7: Pass: 0.326 Bet: 0.674
  7b: Pass: 0.0 Bet: 1.0
  7p: Pass: 0.0 Bet: 1.0
 7pb: Pass: 0.0 Bet: 1.0
   8: Pass: 0.0308 Bet: 0.9692
  8b: Pass: 0.0 Bet: 1.0
  8p: Pass: 0.0 Bet: 1.0
 8pb: Pass: 0.0001 Bet: 0.9999
   9: Pass: 0.4605 Bet: 0.5395
  9b: Pas

These are the results for 1 million training iterations.

It's very interesting to interpret the strategy.
While the exercise in the paper was only for 3 cards, we can increase up to an arbitrary amount.

Similar to real poker, you can see that bluffs tend to come from bottom of range, and the absolute top of range "traps" at a decent frequency, at least on the first action.

In a similar game, with just one street, you would want to be bluffing 1/4 of the time, meaning you need 1 combo of bluff for every 2 combos of value. However, we can see that on the first street, we only have 0.8 combos of bluffs, coming from 1 and 2, whereas we have about 3.2 combos of value. This means we are bluffing 0.8/(0.8+3.2) = 0.8/4 = 1/5 of the time, less than the 1/4th called for.

One reason this could be happening is that, with the existence of three streets, the implied odds artificially increase the size of the pot, so my 1/2 pot size bet (1 into a pot of 2) is actually smaller because of how large my bets will be in comparison to pot size later on. Having bluffs 1/5th of the time is appropriate for a 1/3 pot size bet, so we can approximate this to be the aritificial pot size accounting for future streets. Very cool.